In [2]:
import numpy as np

a, b, c, d, e, f, g = 4/3, 1, 1, 1, 1, 1, 2/3

THIRD = 1/3

# -------- iterative approach 

for i in range(700):
    temp_a = a
    temp_b = b
    temp_c = c
    temp_d = d
    temp_e = e
    temp_f = f
    temp_g = g

    a = THIRD * temp_b
    b = THIRD * temp_a
    c = THIRD * temp_a + temp_c
    d = THIRD * temp_a + temp_d
    e = THIRD * temp_b + temp_e
    f = THIRD * temp_b + temp_f

print(f"i: {i}, a: {a}, b: {b}, c: {c}, d: {d}, e: {e}, f: {f}, g: {g}, sum: {a + b + c + d + e + f + g}")

i: 699, a: 0.0, b: 0.0, c: 1.6249999999999998, d: 1.6249999999999998, e: 1.5416666666666663, f: 1.5416666666666663, g: 0.6666666666666666, sum: 6.999999999999999


In [3]:
# ---------------- Sys of lin eq. approach

import numpy as np
from scipy.optimize import linprog

def compute_steady_state_lp(graph, initial_power):
    """
    Compute the steady-state power distribution using Linear Programming (LP).

    Parameters:
        graph (numpy.ndarray): Adjacency matrix where graph[i][j] is the fraction of
                              power delegated from j to i.
        initial_power (numpy.ndarray): Initial power distribution for each node.

    Returns:
        numpy.ndarray: Steady-state power distribution.
    """
    n = graph.shape[0]  # Number of nodes

    # Objective: Minimize zero vector (we are only solving for constraints)
    c = np.zeros(n)

    # Equality constraints: (I - T) * p = 0
    # where T is the transpose of the adjacency matrix
    A_eq = np.eye(n) - graph.T
    b_eq = np.zeros(n)

    # Additional constraint: sum(p) = sum(initial_power)
    A_eq = np.vstack([A_eq, np.ones(n)])
    b_eq = np.append(b_eq, np.sum(initial_power))

    # Bounds: p[i] >= 0 for all i
    bounds = [(0, None) for _ in range(n)]

    # Solve the linear program
    result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")

    if result.success:
        return result.x
    else:
        raise ValueError("Linear programming did not converge.")



# Example graph: adjacency matrix
graph = np.array([
    [0, THIRD, 0, 0, 0, 0, THIRD],
    [THIRD, 0, 0, 0, 0, 0, 0],  
    [THIRD, 0, 1, 0, 0, 0, 0],
    [THIRD, 0, 0, 1, 0, 0, 0],
    [0, THIRD, 0, 0, 1, 0, 0],
    [0, THIRD, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 2 * THIRD]
])

# Initial power distribution: all nodes start with power 1
initial_power = np.ones(graph.shape[0])

steady_state = compute_steady_state_lp(graph, initial_power)
print("Steady-state power distribution:", steady_state)
print("Sum of steady-state power:", np.sum(steady_state))




ModuleNotFoundError: No module named 'scipy'

In [ ]:
# ---- Monte carlo
# does not work, since e.g. the node G which has 2/3 power and gives 1/3 here kind of does not work. Ill work on this 
# for fun later, its not really relevant to the paper
import random
from collections import defaultdict

def monte_carlo_random_walk(graph, weights, iterations=1000):
    """
    Perform a Monte Carlo simulation on a weighted multidigraph.

    Parameters:
        graph (dict): A dictionary where keys are nodes and values are lists of neighbors.
        weights (dict): A dictionary of edges to weights, e.g., {(u, v): weight}.
        iterations (int): Number of random walks to perform.

    Returns:
        dict: A dictionary with nodes as keys and the proportion of visits as values.
    """
    # Initialize visit counts
    visit_counts = defaultdict(int)

    for key in graph.keys():
        visit_counts[key] = 0

    
    # Get all nodes in the graph
    nodes = list(graph.keys())

    for _ in range(iterations):
        # Start at a random node with uniform probability
        current_node = random.choice(nodes)

        while True:
            # Get neighbors and their corresponding edge weights
            neighbors = graph[current_node]
            edge_weights = [weights[(current_node, neighbor)] for neighbor in neighbors]

            # Normalize weights to get probabilities
            total_weight = sum(edge_weights)
            probabilities = [w / total_weight for w in edge_weights]

            # Choose the next node based on the probabilities
            next_node = random.choices(neighbors, weights=probabilities)[0]
            if next_node == current_node:
                visit_counts[current_node] += 1
                break

            current_node = next_node

    
    # Convert visit counts to proportions
    total_visits = sum(visit_counts.values())
    visit_proportions = {node: count / total_visits for node, count in visit_counts.items()}

    return visit_proportions

# Example usage:
# Define the graph as an adjacency list and the weights for edges
graph = {
    'A': ['B', 'C', 'D'],
    'B': ['A', 'E', 'F'],
    'C': ['C'],
    'D': ['D'],
    'E': ['E'],
    'F': ['F'],
    'G': ['A', 'G'],
}

weights = {
    ('A', 'B'): THIRD,
    ('A', 'C'): THIRD,
    ('A', 'D'): THIRD,
    ('B', 'A'): THIRD,
    ('B', 'E'): THIRD,
    ('B', 'F'): THIRD,
    ('C', 'C'): 1,
    ('D', 'D'): 1,
    ('E', 'E'): 1,
    ('F', 'F'): 1,
    ('G', 'A'): 1,
}

# Run the simulation
result = monte_carlo_random_walk(graph, weights, iterations=10000)
print(result)


KeyError: ('G', 'G')

In [ ]:
# ---- iterative power propagation

THIRD = 1/3

p_A, p_B, p_C, p_D, p_E, p_F, p_G = 0, 0, 0, 0, 0, 0, 0

num_iterations = 20

for i in range (num_iterations):
    #take a snapshot of the current power distribution
    curr_A = p_A
    curr_B = p_B
    curr_C = p_C
    curr_D = p_D
    curr_E = p_E
    curr_F = p_F
    curr_G = p_G

    # each iteration, add one more power to each person, and do the delegations
    p_A = p_A + 1
    p_B = p_B + 1 
    p_C = p_C + 1
    p_D = p_D + 1
    p_E = p_E + 1
    p_F = p_F + 1
    p_G = p_G + 1

    # delegation G-1/3->A
    p_A += curr_G * THIRD
    p_G -= curr_G * THIRD

    # delegation A-1/3->B
    p_B += curr_A * THIRD
    p_A -= curr_A * THIRD

    # delegation A-1/3->C
    p_C += curr_A * THIRD
    p_A -= curr_A * THIRD

    # delegation A-1/3->D
    p_D += curr_A * THIRD
    p_A -= curr_A * THIRD

    # delegation B-1/3->A
    p_A += curr_B * THIRD
    p_B -= curr_B * THIRD

    # delegation B-1/3->E
    p_E += curr_B * THIRD
    p_B -= curr_B * THIRD

    # delegation B-1/3->F
    p_F += curr_B * THIRD
    p_B -= curr_B * THIRD


print("1:")
print(p_A / num_iterations, 
p_B / num_iterations, 
p_C / num_iterations,
p_D / num_iterations,
p_E / num_iterations,
p_F / num_iterations,
p_G / num_iterations)

print(f"sum: {p_A / num_iterations + p_B / num_iterations + p_C / num_iterations + p_D / num_iterations + p_E / num_iterations + p_F / num_iterations + p_G / num_iterations}")

print("Normalized:")
print((p_A / num_iterations) / 7, 
(p_B / num_iterations) / 7, 
(p_C / num_iterations) / 7,
(p_D / num_iterations) / 7,
(p_E / num_iterations) / 7,
(p_F / num_iterations) / 7,
(p_G / num_iterations) / 7)

# ---- iterative power propagation with no source effect

p_A, p_B, p_C, p_D, p_E, p_F, p_G = 1, 1, 1, 1, 1, 1, 1

num_iterations = 20

for i in range (num_iterations):
    #take a snapshot of the current power distribution
    curr_A = p_A
    curr_B = p_B
    curr_C = p_C
    curr_D = p_D
    curr_E = p_E
    curr_F = p_F
    curr_G = p_G

    # delegation G-1/3->A
    p_A += curr_G * THIRD
    p_G -= curr_G * THIRD

    # delegation A-1/3->B
    p_B += curr_A * THIRD
    p_A -= curr_A * THIRD

    # delegation A-1/3->C
    p_C += curr_A * THIRD
    p_A -= curr_A * THIRD

    # delegation A-1/3->D
    p_D += curr_A * THIRD
    p_A -= curr_A * THIRD

    # delegation B-1/3->A
    p_A += curr_B * THIRD
    p_B -= curr_B * THIRD

    # delegation B-1/3->E
    p_E += curr_B * THIRD
    p_B -= curr_B * THIRD

    # delegation B-1/3->F
    p_F += curr_B * THIRD
    p_B -= curr_B * THIRD

print("2")
print(p_A, p_B, p_C, p_D, p_E, p_F, p_G)

print(f"sum: {p_A  + p_B  + p_C  + p_D  + p_E + p_F  + p_G }")

print("Normalized:")
print(p_A / 7,
    p_B / 7,
    p_C / 7,
    p_D / 7,
    p_E / 7,
    p_F / 7,
    p_G / 7)

1:
0.13121992712505542 0.09373496355446154 1.7578425728709113 1.7578425728709113 1.5547025364388165 1.5547025364388165 0.14995489070102677
sum: 7.0
Normalized:
0.01874570387500792 0.013390709079208791 0.25112036755298733 0.25112036755298733 0.22210036234840236 0.22210036234840236 0.021422127243003825
2
0.00020048586881354493 0.000100243077805372 1.8747995141670364 1.8747995141670364 1.6248997570297439 1.6248997570297439 0.0003007286598217179
sum: 7.000000000000001
Normalized:
2.8640838401934988e-05 1.4320439686481716e-05 0.2678285020238623 0.2678285020238623 0.23212853671853484 0.23212853671853484 4.296123711738827e-05


In [ ]:
# ---- Unraveling approach with LE and LP
import sys, os

sys.path.append(os.path.abspath("/Users/DavidHolzwarth/Uni/EPFL/Thesis"))

from LP import matrix_builder_and_solver, build_matrix, solve_linear_programming, solve_linear_equations

delegations = {
    # A
    0: {1: THIRD,
        2: THIRD,
        3: THIRD},   
    # B 
    1: {0: THIRD,
        4: THIRD,
        5: THIRD},
    # C
    2: {2: 1},
    # D
    3: {3: 1},
    # E
    4: {4: 1},
    # F
    5: {5: 1},
    # G
    6: {0: THIRD,
        6: 2 * THIRD}
}
nodes = [0, 1, 2, 3, 4, 5, 6]

A_eq, b_eq = build_matrix(nodes, delegations, 200, mode="SE")
time_lp_se = %timeit -o solve_linear_programming(A_eq, b_eq, len(nodes))
time_le_se = %timeit -o solve_linear_equations(A_eq, b_eq, len(nodes))

A_eq, b_eq = build_matrix(nodes, delegations, 200, mode="NSE")
time_lp_nse = %timeit -o solve_linear_programming(A_eq, b_eq, len(nodes))
time_le_nse = %timeit -o solve_linear_equations(A_eq, b_eq, len(nodes))

print("\n\n\n")
A_eq, b_eq = build_matrix(nodes, delegations, 200, mode="SE")
solve_linear_programming(A_eq, b_eq, len(nodes))
print("Time LP SE:", time_lp_se)
solve_linear_equations(A_eq, b_eq, len(nodes))
print("Time LE SE:", time_le_se)

A_eq, b_eq = build_matrix(nodes, delegations, 200, mode="NSE")
solve_linear_programming(A_eq, b_eq, len(nodes))
print("Time LP NSE:", time_lp_nse)
solve_linear_equations(A_eq, b_eq, len(nodes))
print("Time LE NSE:", time_le_nse)

Normalized LP results:
0.0019, 0.0013, 0.2662, 0.2662, 0.2311, 0.2311, 0.0021,
Normalized LP results:
0.0019, 0.0013, 0.2662, 0.2662, 0.2311, 0.2311, 0.0021,
Normalized LP results:
0.0019, 0.0013, 0.2662, 0.2662, 0.2311, 0.2311, 0.0021,
Normalized LP results:
0.0019, 0.0013, 0.2662, 0.2662, 0.2311, 0.2311, 0.0021,
Normalized LP results:
0.0019, 0.0013, 0.2662, 0.2662, 0.2311, 0.2311, 0.0021,
Normalized LP results:
0.0019, 0.0013, 0.2662, 0.2662, 0.2311, 0.2311, 0.0021,
Normalized LP results:
0.0019, 0.0013, 0.2662, 0.2662, 0.2311, 0.2311, 0.0021,
Normalized LP results:
0.0019, 0.0013, 0.2662, 0.2662, 0.2311, 0.2311, 0.0021,
1.37 s ± 29.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Normalized LE results:
0.0019, 0.0013, 0.2662, 0.2662, 0.2311, 0.2311, 0.0021,
Normalized LE results:
0.0019, 0.0013, 0.2662, 0.2662, 0.2311, 0.2311, 0.0021,
Normalized LE results:
0.0019, 0.0013, 0.2662, 0.2662, 0.2311, 0.2311, 0.0021,
Normalized LE results:
0.0019, 0.0013, 0.2662, 0.2662, 0.2311, 

In [1]:
import sys, os
sys.path.append(os.path.abspath("/Users/DavidHolzwarth/Uni/EPFL/bachelors-thesis"))
from LP import resolve_delegations_LP, invert_graph
from graph_viz import visualize_delegation_graph

THIRD = 1/3

delegations = {
    # A
    'A': {'B': THIRD,
        'C': THIRD,
        'D': THIRD},   
    # B 
    'B': {'A': THIRD,
        'E': THIRD,
        'F': THIRD},
    # G
    'G': {'A': THIRD,
        'G': 2 * THIRD}
}

nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

print(resolve_delegations_LP(invert_graph(delegations), nodes))

visualize_delegation_graph(delegations, nodes)


OrderedDict({'Constraint_A': 1*A + -0.3333333333333333*B + -0.3333333333333333*G + -1.0 = 0, 'Constraint_B': -0.3333333333333333*A + 1*B + -1.0 = 0, 'Constraint_C': -0.3333333333333333*A + 1*C + -1.0 = 0, 'Constraint_D': -0.3333333333333333*A + 1*D + -1.0 = 0, 'Constraint_E': -0.3333333333333333*B + 1*E + -1.0 = 0, 'Constraint_F': -0.3333333333333333*B + 1*F + -1.0 = 0, 'Constraint_G': 0.33333333333333337*G + -1.0 = 0, 'SinkNodesConstraint': 1*C + 1*D + 1*E + 1*F + -7 = 0})
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/thesis/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/q9/0xq_2r1503z58vb1xx__4r5m0000gn/T/9090cf21b23e4f57abed283a3a9a99ab-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/q9/0xq_2r1503z58vb1xx__4r5m0000gn/T/9090cf21b23e4f57abed283a3a9a99ab-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 33 RHS
At lin

/opt/anaconda3/envs/thesis/lib/python3.12/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


No trigger
